In [1]:
import os
import re

from dotenv import load_dotenv
from langchain import hub
from langchain.agents import AgentType, load_tools
from langchain_experimental.tools import PythonREPLTool
from langchain.agents.initialize import initialize_agent
from langchain.agents import create_react_agent, AgentExecutor
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import JSONAgentOutputParser
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain_openai import ChatOpenAI
from langchain.tools import ArxivQueryRun, WikipediaQueryRun
from langchain.tools.render import render_text_description_and_args
from langchain.utilities import ArxivAPIWrapper, WikipediaAPIWrapper
from langchain.prompts import MessagesPlaceholder

# from langchain.schema import ChatMessage, SystemMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.base import BaseCallbackManager

from llamp.mp.agents import (
    MPSummaryExpert,
    MPThermoExpert,
    MPElasticityExpert,
    MPDielectricExpert,
    MPPiezoelectricExpert,
    MPMagnetismExpert,
    MPElectronicExpert,
    MPSynthesisExpert,
    MPStructureRetriever,
)
# from llamp.arxiv.agents import ArxivAgent

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", None)
OPENAI_ORGANIZATION = os.getenv("OPENAI_ORGANIZATION", None)

OPENAI_GPT_MODEL = "gpt-4-1106-preview"
# OPENAI_GPT_MODEL = "gpt-4-0125-preview"
# OPENAI_GPT_MODEL = "gpt-3.5-turbo-1106"
# OPENAI_GPT_MODEL = "gpt-4"

No module named 'phonopy'
No module named 'phonopy'


In [2]:
top_llm = ChatOpenAI(
    temperature=0.1,
    model=OPENAI_GPT_MODEL,
    openai_api_key=OPENAI_API_KEY,
    openai_organization=OPENAI_ORGANIZATION,
    streaming=False,
    callbacks=[StreamingStdOutCallbackHandler()],
)

bottom_callback_handler = StreamingStdOutCallbackHandler()

bottom_llm = ChatOpenAI(
    temperature=0,
    model=OPENAI_GPT_MODEL,
    openai_api_key=OPENAI_API_KEY,
    openai_organization=OPENAI_ORGANIZATION,
    max_retries=5,
    streaming=True,
    callbacks=[bottom_callback_handler],
)


wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
arxiv = ArxivQueryRun(api_wrapper=ArxivAPIWrapper())

tools = load_tools(["llm-math"], llm=bottom_llm)
tools += [PythonREPLTool()]
tools += [
    MPThermoExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPElasticityExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPDielectricExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPMagnetismExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPElectronicExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPPiezoelectricExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPSummaryExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPSynthesisExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPStructureRetriever(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    # ArxivAgent(llm=bottom_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    arxiv,
    wikipedia,
]

instructions = re.sub(
        r"\s+",
        " ",
        """You are a data-aware agent that can consult materials-related
    data through Materials Project (MP) database, arXiv, Wikipedia, and a python 
    REPL, which you can use to execute python code. If you get an error, debug 
    your code and try again. Only use the output of your code to answer the 
    question. Ask user to clarify their queries if needed. Please note that you 
    don't have direct control over MP but through multiple assistant agents to 
    help you. You need to provide complete context in the input for assistants to 
    do their job.
    """,
    ).replace("\n", " ")
base_prompt = hub.pull("langchain-ai/react-agent-template")
prompt = base_prompt.partial(instructions=instructions)

conversational_memory = ConversationBufferWindowMemory(
    memory_key="chat_history", k=5, return_messages=True
)

agent = create_react_agent(top_llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True,
    handle_parsing_errors=True,
    memory=conversational_memory,
)


In [3]:
response = agent_executor.invoke(
    {
        "input": """Recommand the synthesis recipe for BaTiO3 and calculate the reaction enthalpy for each step.""",
    }
)



> Entering new AgentExecutor chain...
To recommend a synthesis recipe for BaTiO3, I will use the MPSynthesisExpert tool to access the Materials Project synthesis endpoint. After obtaining the synthesis recipe, I will then calculate the reaction enthalpy for each step using the MPThermoExpert tool.

Let's start by finding the synthesis recipe for BaTiO3.

Action: MPSynthesisExpert
Action Input: BaTiO3To recommend a synthesis recipe for BaTiO3, I will use the MPSynthesisExpert tool to access the Materials Project synthesis endpoint. After obtaining the synthesis recipe, I will then calculate the reaction enthalpy for each step using the MPThermoExpert tool.

Let's start by finding the synthesis recipe for BaTiO3.

Action: MPSynthesisExpert
Action Input: BaTiO3

> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Final Answer",
  "action_input": "Please provide more context or specify a question regarding Barium Titanate (BaTiO3)."
}
```Action:
```
{
  "action": "Final Answ

Retrieving SynthesisSearchResultModel documents:   0%|          | 0/314 [00:00<?, ?it/s]

[{'doi': '10.1016/j.jcrysgro.2017.11.003', 'paragraph_string': 'All chemicals were used without further treatment for the synthesis. BaCO3 (≥99%, Solvay), TiO2 (≥...', 'synthesis_type': 'solid-state', 'reaction_string': '1 BaCO3 + 1 TiO2 == 1 BaTiO3 + 1 CO2', 'reaction': {'left_side': [{'amount': '1', 'material': 'TiO2'}, {'amount': '1', 'material': 'BaCO3'}], 'right_side': [{'amount': '1', 'material': 'BaTiO3'}, {'amount': '1', 'material': 'CO2'}]}, 'targets_formula': ['Ba1 Ti1 O3'], 'target': {'material_string': 'BaTiO3', 'material_name': '', 'material_formula': 'BaTiO3', 'phase': None, 'is_acronym': False, 'composition': [{'formula': 'BaTiO3', 'amount': '1', 'elements': {'Ba': '1', 'Ti': '1', 'O': '3'}}], 'amounts_vars': {}, 'elements_vars': {}, 'additives': [], 'oxygen_deficiency': 'None'}, 'targets_formula_s': ['BaTiO3'], 'precursors_formula_s': ['TiO2', 'BaCO3'], 'precursors': [{'material_string': 'TiO2', 'material_name': '', 'material_formula': 'TiO2', 'phase': None, 'is_acronym

Retrieving ThermoDoc documents:   0%|          | 0/186 [00:00<?, ?it/s]

[{'formula_pretty': 'TiO2', 'material_id': 'mp-766454', 'formation_energy_per_atom': -3.276080316666668}, {'formula_pretty': 'TiO2', 'material_id': 'mp-1439', 'formation_energy_per_atom': -3.3174958547222233}, {'formula_pretty': 'TiO2', 'material_id': 'mp-1840', 'formation_energy_per_atom': -3.3189781280555564}, {'formula_pretty': 'TiO2', 'material_id': 'mp-9173', 'formation_energy_per_atom': -3.264519665555555}, {'formula_pretty': 'TiO2', 'material_id': 'mp-1201727', 'formation_energy_per_atom': -3.255430827847222}, {'formula_pretty': 'TiO2', 'material_id': 'mp-390', 'formation_energy_per_atom': -3.3220194555555564}, {'formula_pretty': 'TiO2', 'material_id': 'mp-554278', 'formation_energy_per_atom': -3.315602058055557}, {'formula_pretty': 'TiO2', 'material_id': 'mp-775938', 'formation_energy_per_atom': -3.3097725644444456}, {'formula_pretty': 'TiO2', 'material_id': 'mp-754769', 'formation_energy_per_atom': -3.2857260580555554}, {'formula_pretty': 'TiO2', 'material_id': 'mp-756655', 'f

...

[{'formula_pretty': 'TiO2', 'material_id': 'mp-766454', 'formation_energy_per_atom': -3.276080316666668}, {'formula_pretty': 'TiO2', 'material_id': 'mp-1439', 'formation_energy_per_atom': -3.3174958547222233}, {'formula_pretty': 'TiO2', 'material_id': 'mp-1840', 'formation_energy_per_atom': -3.3189781280555564}, {'formula_pretty': 'TiO2', 'material_id': 'mp-9173', 'formation_energy_per_atom': -3.264519665555555}, {'formula_pretty': 'TiO2', 'material_id': 'mp-1201727', 'formation_energy_per_atom': -3.255430827847222}, {'formula_pretty': 'TiO2', 'material_id': 'mp-390', 'formation_energy_per_atom': -3.3220194555555564}, {'formula_pretty': 'TiO2', 'material_id': 'mp-554278', 'formation_energy_per_atom': -3.315602058055557}, {'formula_pretty': 'TiO2', 'material_id': 'mp-775938', 'formation_energy_per_atom': -3.3097725644444456}, {'formula_pretty': 'TiO2', 'material_id': 'mp-754769', 'formation_energy_per_atom': -3.2857260580555554}, {'formula_pretty': 'TiO2', 'material_id': 'mp-756655', 'f

Retrieving ThermoDoc documents:   0%|          | 0/28 [00:00<?, ?it/s]

[{'formula_pretty': 'BaTiO3', 'material_id': 'mp-558125', 'formation_energy_per_atom': -3.3710064950000023}, {'formula_pretty': 'BaTiO3', 'material_id': 'mp-504715', 'formation_energy_per_atom': -0.41220425916666625}, {'formula_pretty': 'BaTiO3', 'material_id': 'mp-5986', 'formation_energy_per_atom': -3.370346751333332}, {'formula_pretty': 'BaTiO3', 'material_id': 'mp-5777', 'formation_energy_per_atom': -3.372303325333334}, {'formula_pretty': 'BaTiO3', 'material_id': 'mp-5020', 'formation_energy_per_atom': -3.3722623353333345}, {'formula_pretty': 'BaTiO3', 'material_id': 'mp-5933', 'formation_energy_per_atom': -3.3696838833333347}, {'formula_pretty': 'BaTiO3', 'material_id': 'mp-2998', 'formation_energy_per_atom': -3.364836653333333}, {'formula_pretty': 'BaTiO3', 'material_id': 'mp-644497', 'formation_energy_per_atom': -2.7754237491666656}, {'formula_pretty': 'BaTiO3', 'material_id': 'mp-5933', 'formation_energy_per_atom': -3.487076905166634}, {'formula_pretty': 'BaTiO3', 'material_id'

Retrieving ThermoDoc documents:   0%|          | 0/30 [00:00<?, ?it/s]

[{'formula_pretty': 'CO2', 'material_id': 'mp-556034', 'thermo_id': 'mp-556034_R2SCAN', 'formation_energy_per_atom': -1.2950058350000002}, {'formula_pretty': 'CO2', 'material_id': 'mp-20066', 'thermo_id': 'mp-20066_R2SCAN', 'formation_energy_per_atom': -1.3055174775000005}, {'formula_pretty': 'CO2', 'material_id': 'mp-1102227', 'thermo_id': 'mp-1102227_GGA_GGA+U_R2SCAN', 'formation_energy_per_atom': -1.749815679166667}, {'formula_pretty': 'CO2', 'material_id': 'mp-995224', 'thermo_id': 'mp-995224_GGA_GGA+U_R2SCAN', 'formation_energy_per_atom': -1.4691622708333334}, {'formula_pretty': 'CO2', 'material_id': 'mp-1077906', 'thermo_id': 'mp-1077906_R2SCAN', 'formation_energy_per_atom': -1.298242574166667}, {'formula_pretty': 'CO2', 'material_id': 'mp-1190699', 'thermo_id': 'mp-1190699_R2SCAN', 'formation_energy_per_atom': -1.2981332966666663}, {'formula_pretty': 'CO2', 'material_id': 'mp-995198', 'thermo_id': 'mp-995198_GGA_GGA+U_R2SCAN', 'formation_energy_per_atom': -1.4480303691666663}, {

In [5]:
agent_executor.memory.chat_memory.messages[1].content

'The reaction enthalpy for the synthesis of BaTiO3 from BaCO3 and TiO2 is approximately 1.9252 eV.'